In [55]:
#!pip install unipath
#!pip install pyarrow

In [24]:
import os
import pandas as pd
import glob
from unipath import Path

In [65]:
#현재 파일의 상위 폴더 위치
PARENT_DIR = os.path.dirname(os.getcwd())
# CSV FILE
CSV_FOLDER = os.path.join(PARENT_DIR,'csv_files')

#Test FOLDER
TEST_FOLDER = os.path.join(PARENT_DIR,'test_files')
# PQ FILE
PQ_FOLDER = os.path.join(PARENT_DIR,'pq_files')

#ANALYSIS
ANALYSIS_FOLDER = os.path.join(PARENT_DIR,'analysis')

os.makedirs(ANALYSIS_FOLDER,exist_ok=True)
os.makedirs(TEST_FOLDER,exist_ok=True)
#컬럼 염문명
new_cols = ['voting_date','ranking','title','released_date','sales',
            'sales_market_portion','sales_comp_yesterday','sales_fluctuation_rate',
            'total_sales','audience_num','audience_num_delta',
            'audience_num_fluctuatation_rate','total_audience','screen_num',
            'total_played_time','country_of_origin','country','production_comp_name',
            'distributor_name','film_ratings','genre','director','performers']

csv_files = glob.glob(os.path.join(CSV_FOLDER,'*.csv'))

In [68]:
df_combined.shape

(211646, 23)

In [69]:
df_combined.to_csv(os.path.join(TEST_FOLDER,'combined.csv'),header=True,index=False,encoding='utf-8-sig')
df_combined.to_parquet(os.path.join(TEST_FOLDER,'combined.parquet'),engine='pyarrow',compression='snappy')

In [59]:
# CSV 파일 뭉치기
df_combined = pd.DataFrame()
csv_files = glob.glob(os.path.join(CSV_FOLDER,'*.csv'))

for file in csv_files:
    df = pd.read_csv(file)
    df_combined  = pd.concat([df_combined,df])



OSError: Cannot save file into a non-existent directory: 'd:\DevCourse\Proj2\Proj2\test_files'

In [56]:
#컬럼명 한글에서 영어로 변경
for file in csv_files:
    df = pd.read_csv(file)
    file_path = Path(file)
    df.columns = new_cols
    fname = str(file_path.name)
    df.to_csv(os.path.join(CSV_FOLDER,fname[:-4]+'.csv'),header=True,index=False,encoding='utf-8-sig')
    df.to_parquet(os.path.join(PQ_FOLDER,fname[:-4]+'.parquet'),engine='pyarrow',compression='snappy') #추출한 날짜를 파일명으로 하고 Parquet으로 저장

In [50]:
# 연도와 장르 1대1 mapping 하기
df_fin = pd.DataFrame()
for file in csv_files:
    df = pd.read_csv(file)
    df2 = df[['voting_date','genre','audience_num']]
    df2['genre'] = df2['genre'].str.split(',')
    df2 = df2.explode("genre")
    df2["genre"] = df2["genre"].str.strip()
    df2.dropna(inplace=True)
    df_fin = pd.concat([df_fin,df2])


C:\Users\user\AppData\Local\Temp\ipykernel_27560\3344201676.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['genre'] = df2['genre'].str.split(',')
C:\Users\user\AppData\Local\Temp\ipykernel_27560\3344201676.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['genre'] = df2['genre'].str.split(',')
C:\Users\user\AppData\Local\Temp\ipykernel_27560\3344201676.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [51]:
df_fin.to_csv(os.path.join(ANALYSIS_FOLDER,'genre_by_date.csv'),header=True,index=False,encoding='utf-8-sig')